In [15]:
# Data Set voorbereiden
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import pyodbc

DBAdventureWorks = {"servername": "localhost\SQLEXPRESS",
      "database": "AdventureWorks2019"}
DBaenc = {"servername": "localhost\SQLEXPRESS",
      "database": "aenc"}
DBNorthwind = {"servername": "localhost\SQLEXPRESS",
      "database": "Northwind"}


export_connAdventureWorks = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBAdventureWorks['servername'] + 
                             ';DATABASE=' + DBAdventureWorks['database'] + ';Trusted_Connection=yes')
export_connaenc = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBaenc['servername'] + 
                             ';DATABASE=' + DBaenc['database'] + ';Trusted_Connection=yes')
export_connNorthwind = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBNorthwind['servername'] + 
                             ';DATABASE=' + DBNorthwind['database'] + ';Trusted_Connection=yes')

export_cursorAdventureWorks = export_connAdventureWorks.cursor()
export_cursoraenc = export_connaenc.cursor()
export_cursorNorthwind = export_connNorthwind.cursor()








# Historische verkoopdata selecteren

In [53]:
adventureworks_data = pd.read_sql_query("SELECT * FROM Sales.SalesOrderDetail;", export_connAdventureWorks)
AdventurWorksSalesOrderHeader = pd.read_sql_query("SELECT * FROM Sales.SalesOrderHeader", export_connAdventureWorks)
adventureworks_data = adventureworks_data.merge(AdventurWorksSalesOrderHeader, on='SalesOrderID')
adventureworks_data['Source'] = 'AdventureWorks2019'

aenc_data = pd.read_sql_query("SELECT * FROM sales_order_item",export_connaenc)
aenc_data = aenc_data.rename(columns={'prod_id': 'ProductID', 'quantity': 'OrderQty', 'ship_date': 'OrderDate'})
aenc_data['Source'] = 'aenc'

northwind_data = pd.read_sql_query("select * from dbo.[Order Details]",export_connNorthwind)
orders_data = pd.read_sql_query("SELECT * FROM Orders", export_connNorthwind)

northwind_data = northwind_data.merge(orders_data, on='OrderID')
northwind_data = northwind_data.rename(columns={'Quantity': 'OrderQty'})
northwind_data['Source'] = 'Northwind'



,id,line_id,ProductID,OrderQty,OrderDate,Source
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,aenc
1,2001,2,301,12,14-Sep-1996 12:00:00 AM,aenc
2,2001,3,302,12,14-Sep-1996 12:00:00 AM,aenc
3,2002,1,400,24,18-Sep-1996 12:00:00 AM,aenc
4,2002,2,401,24,18-Sep-1996 12:00:00 AM,aenc
...,...,...,...,...,...,...
1098,2652,2,300,10,01-Dec-1998 12:00:00 AM,aenc
1099,2653,1,301,0,17-Oct-1999 12:00:00 AM,aenc
1100,2654,1,301,15,29-Oct-1999 12:00:00 AM,aenc
1101,2654,2,302,3,29-Oct-1999 12:00:00 AM,aenc


# Datasets combineren

In [55]:
combined_data = pd.concat([adventureworks_data[['ProductID', 'OrderQty', 'OrderDate', 'Source']], 
                           aenc_data[['ProductID', 'OrderQty', 'OrderDate', 'Source']], 
                           northwind_data[['ProductID', 'OrderQty', 'OrderDate', 'Source']]], ignore_index=True)

pd.set_option('display.max_rows',50)
combined_data

,ProductID,OrderQty,OrderDate,Source
0,776,1,2011-05-31 00:00:00,AdventureWorks2019
1,777,3,2011-05-31 00:00:00,AdventureWorks2019
2,778,1,2011-05-31 00:00:00,AdventureWorks2019
3,771,1,2011-05-31 00:00:00,AdventureWorks2019
4,772,1,2011-05-31 00:00:00,AdventureWorks2019
...,...,...,...,...
124570,64,2,1998-05-06 00:00:00,Northwind
124571,66,1,1998-05-06 00:00:00,Northwind
124572,73,2,1998-05-06 00:00:00,Northwind
124573,75,4,1998-05-06 00:00:00,Northwind


# Data voorbereiden

In [52]:
# Verwijder missende waarden
combined_data.dropna(inplace=True)

# Converteer OrderDate naar datetime
combined_data['OrderDate'] = pd.to_datetime(combined_data['OrderDate'])

# Voeg extra features toe zoals jaar en maand
combined_data['Year'] = combined_data['OrderDate'].dt.year
combined_data['Month'] = combined_data['OrderDate'].dt.month

# Bereken verkoopvolume per productcategorie per maand
aggregated_data = combined_data.groupby(['ProductID', 'Year', 'Month']).agg({'OrderQty': 'sum', 'Source': 'first'}).reset_index()
aggregated_data.rename(columns={'OrderQty': 'TotalOrderQty'}, inplace=True)

pd.set_option('display.max_rows', 50)
aggregated_data

,ProductID,Year,Month,TotalOrderQty,Source
0,1,1996,8,63,Northwind
1,1,1996,9,20,Northwind
2,1,1996,11,27,Northwind
3,1,1996,12,15,Northwind
4,1,1997,1,34,Northwind
...,...,...,...,...,...
5245,700,1998,7,24606072364848481212,aenc
5246,700,1998,8,6048601212364824,aenc
5247,700,1998,9,12602424123612,aenc
5248,700,1998,10,364836364860,aenc
